In [1]:
# diagnostico_activos.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

# -------- CONFIG ----------
INPUT_CSV = 'Asset_Inventory_-_Public_20251119.csv'
OUTPUT_DIR = './diagnostico_output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# columnas que consideramos metadatos críticos (ajusta según tu esquema)
CRITICAL_COLUMNS = [
    'dataset_id', 'title', 'description', 'theme', 'publisher',
    'access_url', 'format', 'last_updated'
]

# -------- 1) Cargar datos ----------
df = pd.read_csv(INPUT_CSV, dtype=str)  # leer todo como str inicialmente
df_orig = df.copy()
print("Filas cargadas:", len(df))
print("Columnas:", list(df.columns)[:20])

# Normalizar nombres de columnas (minúsculas, sin espacios)
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# mapear nombres comunes a nuestros nombres esperados
col_map = {}
# ejemplos de mapeo (añade si tu CSV tiene nombres distintos)
col_map_candidates = {
    'title': ['title', 'name', 'dataset_name'],
    'description': ['description', 'abstract', 'summary'],
    'theme': ['theme', 'category', 'topic', 'subject'],
    'publisher': ['publisher', 'owner', 'agency'],
    'access_url': ['access_url', 'url', 'download_url', 'link'],
    'format': ['format', 'file_type', 'data_format'],
    'last_updated': ['last_updated', 'modified', 'updated_at', 'date_updated'],
    'dataset_id': ['id', 'dataset_id', 'resource_id']
}
for target, candidates in col_map_candidates.items():
    for cand in candidates:
        if cand in df.columns:
            col_map[target] = cand
            break

# aplicar renombrado
df = df.rename(columns={v: k for k, v in col_map.items()})
print("Columnas encontradas y usadas:", col_map)

Filas cargadas: 49542
Columnas: ['UID', 'Titulo', 'Descripción', 'Dueño', 'UID del dueño', 'Etapa de publicación', 'Público', 'Estado de aprobación', 'Origen', 'Tipo', 'Categoría', 'Etiqueta', 'url', 'API', 'Fecha de creación (UTC)', 'Fecha de última actualización de metadatos (UTC)', 'Fecha de última actualización de datos (UTC)', 'Dominio', 'Recurso Derivado', 'Vistas']
Columnas encontradas y usadas: {'access_url': 'url'}


In [20]:
df.columns

Index(['uid', 'titulo', 'descripción', 'dueño', 'uid_del_dueño',
       'etapa_de_publicación', 'público', 'estado_de_aprobación', 'origen',
       'tipo', 'categoría', 'etiqueta', 'access_url', 'api',
       'fecha_de_creación_(utc)',
       'fecha_de_última_actualización_de_metadatos_(utc)',
       'fecha_de_última_actualización_de_datos_(utc)', 'dominio',
       'recurso_derivado', 'vistas', 'descargas', 'etiqueta_de_las_filas',
       'número_de_filas', 'número_de_columnas',
       'correo_electrónico_de_contacto', 'licencia', 'atribución',
       'enlace_de_atribución', 'nombre_del_recurso_publicado',
       'uid_del_recurso_publicado', 'uid_del_conjunto_de_datos_superior',
       'información_de_datos:_fecha_emisión_(dd/mm/aaaa)',
       'información_de_datos:_fecha_emisión_(aaaa_mm_dd)',
       'información_de_datos:_frecuencia_de_actualización',
       'información_de_datos:_cobertura_geográfica',
       'información_de_datos:_idioma',
       'información_de_datos:_url_document

In [2]:
# -------- 2) Métricas de completitud ----------
def completeness(series):
    total = len(series)
    non_null = series.dropna().astype(str).replace('', np.nan).dropna()
    return len(non_null) / total if total>0 else np.nan

completitud_por_campo = {}
for col in df.columns:
    completitud_por_campo[col] = completeness(df[col])

comp_df = pd.Series(completitud_por_campo).sort_values(ascending=False)
comp_df.to_csv(os.path.join(OUTPUT_DIR, 'completitud_por_campo.csv'), header=['completitud'])

# completitud por fila (proporción de campos no nulos)
df['__num_present_meta'] = df.notna().sum(axis=1)
df['__pct_present_meta'] = df['__num_present_meta'] / len(df.columns)
df[['__num_present_meta','__pct_present_meta']].to_csv(os.path.join(OUTPUT_DIR, 'completitud_por_fila.csv'))


In [3]:
# -------- 3) Frecuencia de actualización ----------
# intentamos parsear la columna last_updated si existe
if 'last_updated' in df.columns:
    # limpiar strings vacíos
    ser = df['last_updated'].astype(str).replace({'nan': None, '': None})
    # intentar parseo flexible
    def try_parse_date(s):
        if s is None or s.lower() in ['nan','none','']:
            return pd.NaT
        for fmt in ('%Y-%m-%d', '%d-%m-%Y', '%Y/%m/%d', '%d/%m/%Y', '%Y-%m-%dT%H:%M:%S'):
            try:
                return pd.to_datetime(s, format=fmt, errors='raise')
            except Exception:
                pass
        try:
            return pd.to_datetime(s, errors='coerce')
        except:
            return pd.NaT
    parsed = ser.apply(try_parse_date)
    df['last_updated_parsed'] = parsed
    parsed.dropna().to_csv(os.path.join(OUTPUT_DIR, 'last_updated_parsed.csv'), index=False)
else:
    print("No se encontró columna 'last_updated' (intenta renombrar según tu CSV).")


No se encontró columna 'last_updated' (intenta renombrar según tu CSV).


In [4]:
# -------- 4) Cobertura temática ----------
theme_col = None
for candidate in ['theme','category','topic','subject']:
    if candidate in df.columns:
        theme_col = candidate
        break

if theme_col:
    temas = df[theme_col].fillna('Sin tema').astype(str)
    temas_count = temas.value_counts(dropna=False)
    temas_count.to_csv(os.path.join(OUTPUT_DIR, 'cobertura_tematica.csv'))
else:
    print("No se encontró columna temática.")

No se encontró columna temática.


In [5]:
# -------- 5) Visualizaciones ----------
# 5.1 completitud por campo (barras)
plt.figure(figsize=(10,6))
comp_df.plot(kind='bar')
plt.title('Completitud por campo (porcentaje de valores presentes)')
plt.ylabel('Fracción presente (0-1)')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'completitud_por_campo.png'))
plt.close()

In [6]:
# 5.2 top 20 campos más incompletos
plt.figure(figsize=(10,6))
comp_df.sort_values().head(20).plot(kind='barh')
plt.title('Top 20 campos más incompletos')
plt.xlabel('Fracción presente (0-1)')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'top20_incompletos.png'))
plt.close()

In [7]:
# 5.3 histograma de % de metadatos por fila
plt.figure(figsize=(8,5))
df['__pct_present_meta'].hist(bins=20)
plt.title('Distribución de completitud por activo (porcentaje de campos presentes)')
plt.xlabel('Fracción de campos presentes')
plt.ylabel('Número de activos')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'hist_completitud_fila.png'))
plt.close()

In [9]:
# 5.4 si hay fecha, serie temporal de actualizaciones
if 'last_updated_parsed' in df.columns and df['last_updated_parsed'].notna().sum()>0:
    s = df['last_updated_parsed'].dropna().dt.to_period('M').value_counts().sort_index()
    plt.figure(figsize=(10,5))
    s.plot()
    plt.title('Número de activos por mes de última actualización')
    plt.ylabel('Activos')
    plt.xlabel('Mes (AAAA-MM)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'actualizaciones_por_mes.png'))
    plt.close()

In [10]:
# 5.5 cobertura temática plot (top 10)
if theme_col:
    temas_count_head = temas_count.head(10)
    plt.figure(figsize=(8,6))
    temas_count_head.plot(kind='bar')
    plt.title('Top 10 temas por número de activos')
    plt.ylabel('Activos')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'top10_temas.png'))
    plt.close()

In [12]:
# -------- 6) Resumen ejecutivo (archivo HTML simple) ----------
html_parts = []
html_parts.append("<html><head><meta charset='utf-8'><title>Informe diagnóstico</title></head><body>")
html_parts.append(f"<h1>Informe diagnóstico de inventario</h1>")
html_parts.append(f"<p>Archivo analizado: {INPUT_CSV}</p>")
html_parts.append(f"<p>Fecha de generación: {datetime.now().isoformat()}</p>")
# incluir principales tablas y gráficos
html_parts.append("<h2>Completitud por campo (top)</h2>")
html_parts.append("<img src='completitud_por_campo.png' style='max-width:100%;height:auto;'/>")
html_parts.append("<h2>Top 20 campos más incompletos</h2>")
html_parts.append("<img src='top20_incompletos.png' style='max-width:100%;height:auto;'/>")
html_parts.append("<h2>Distribución de completitud por activo</h2>")
html_parts.append("<img src='hist_completitud_fila.png' style='max-width:100%;height:auto;'/>")
if 'last_updated_parsed' in df.columns and df['last_updated_parsed'].notna().sum()>0:
    html_parts.append("<h2>Actualizaciones por mes</h2>")
    html_parts.append("<img src='actualizaciones_por_mes.png' style='max-width:100%;height:auto;'/>")
if theme_col:
    html_parts.append("<h2>Top temas</h2>")
    html_parts.append("<img src='top10_temas.png' style='max-width:100%;height:auto;'/>")

# agregar tabla breve de completitud (top 10 más y menos completos)
html_parts.append("<h2>Tabla: completitud por campo (extracto)</h2>")
html_parts.append(comp_df.round(3).to_frame('completitud').head(20).to_html())

html_parts.append("</body></html>")
html = "\n".join(html_parts)
with open(os.path.join(OUTPUT_DIR, 'informe_diagnostico.html'), 'w', encoding='utf-8') as f:
    f.write(html)

print("Salida generada en:", OUTPUT_DIR)
print("Archivos: ", os.listdir(OUTPUT_DIR))

Salida generada en: ./diagnostico_output
Archivos:  ['completitud_por_campo.csv', 'completitud_por_campo.png', 'completitud_por_fila.csv', 'hist_completitud_fila.png', 'informe_diagnostico.html', 'top20_incompletos.png']


In [21]:
import pandas as pd
import csv

# Ruta del archivo (CAMBIA ESTA LÍNEA)
csv_path = r"C:\Users\Emcali-14\Desktop\MINTIC\Asset_Inventory_-_Public_20251119.csv"

# --- 1. Detectar delimitador automáticamente ---
def detectar_delimitador(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        sample = f.read(5000)
        dialect = csv.Sniffer().sniff(sample, delimiters=[',',';','\t'])
        return dialect.delimiter

delim = detectar_delimitador(csv_path)
print(f"Delimitador detectado: '{delim}'")

# --- 2. Leer el CSV con pandas ---
df = pd.read_csv(csv_path, delimiter=delim, encoding="utf-8")

# --- 3. Mostrar información relevante ---
print("\n=== COLUMNAS ===")
print(df.columns.tolist())

print(f"\nTotal de columnas: {len(df.columns)}")
print(f"Total de filas: {len(df)}")

print("\n=== Primeras 5 filas ===")
print(df.head())


Delimitador detectado: ','

=== COLUMNAS ===
['UID', 'Titulo', 'Descripción', 'Dueño', 'UID del dueño', 'Etapa de publicación', 'Público', 'Estado de aprobación', 'Origen', 'Tipo', 'Categoría', 'Etiqueta', 'url', 'API', 'Fecha de creación (UTC)', 'Fecha de última actualización de metadatos (UTC)', 'Fecha de última actualización de datos (UTC)', 'Dominio', 'Recurso Derivado', 'Vistas', 'Descargas', 'Etiqueta de las Filas', 'Número de Filas', 'Número de Columnas', 'Correo Electrónico de Contacto', 'Licencia', 'Atribución', 'Enlace de Atribución', 'Nombre del Recurso Publicado', 'UID del Recurso Publicado', 'UID del conjunto de datos superior', 'Información de Datos: Fecha Emisión (dd/mm/aaaa)', 'Información de Datos: Fecha Emisión (aaaa mm dd)', 'Información de Datos: Frecuencia de Actualización', 'Información de Datos: Cobertura Geográfica', 'Información de Datos: Idioma', 'Información de Datos: URL Documentación', 'Información de la Entidad: Nombre de la Entidad', 'Información de la En

C:\Users\Emcali-14\AppData\Local\Temp\ipykernel_20600\1943138408.py:18: DtypeWarning: Columns (24,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, delimiter=delim, encoding="utf-8")
